# For BG smartwatch project

# Code Setup {example code, some of it needs revising at main loop}[Also, The directory used in the code is just to my specifc machine, please use your own]

#### 1. Dependency Installation and Imports [Example]

In [ ]:
!pip install pandas

In [ ]:
!pip list

In [ ]:
# !pip install vitaldb pandas numpy matplotlib scipy scikit-learn

import pandas as pd
import vitaldb
import numpy as np
import os
from scipy.signal import find_peaks, butter, lfilter
from scipy.stats import entropy

print("All libraries imported successfully.")



#### 2. Configuration [Example]

This cell sets up all file paths and parameters in a single, easy-to-manage location.

In [ ]:

# --- CONFIGURATION ---
OUTPUT_DIR = "./processed_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "vitaldb_ppg_extracted_features.csv")

VITALDB_DATA_URL = "https://api.vitaldb.net/cases"
VITALDB_TRACKS_URL = "https://api.vitaldb.net/trks"

PPG_SIGNAL_NAME = 'SNUADC/PLETH'
SAMPLE_RATE_HZ = 100
WINDOW_DURATION_SECONDS = 30
SAMPLES_PER_WINDOW = SAMPLE_RATE_HZ * WINDOW_DURATION_SECONDS
VALID_WINDOW_MINUTES = 8  # +/- window around 'opstart' for BG stability

print("Configuration loaded.")



#### 3. Utility Functions [Example]

This cell contains the helper functions for bandpass filtering and extracting hand-engineered features from the PPG signal.

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def extract_ppg_features(ppg_series, fs):
    """
    Extracts a set of robust, hand-engineered features from a single PPG signal window.
    """
    features = {}

    # --- Time-domain features ---
    features['ppg_mean'] = np.mean(ppg_series)
    features['ppg_std'] = np.std(ppg_series)

    # Peak detection and related features
    peaks, _ = find_peaks(ppg_series, distance=50, height=0)
    if len(peaks) > 1:
        # Peak-to-Peak Interval (P-P) in seconds
        pp_intervals = np.diff(peaks)
        features['mean_pp_interval_s'] = np.mean(pp_intervals) / fs
        features['std_pp_interval_s'] = np.std(pp_intervals) / fs
        features['ppg_freq'] = fs / np.mean(pp_intervals)
    else:
        features['mean_pp_interval_s'] = 0
        features['std_pp_interval_s'] = 0
        features['ppg_freq'] = 0

    # Area under the curve (approximated)
    features['auc'] = np.trapz(ppg_series)

    # Derivatives
    derivative = np.diff(ppg_series)
    features['first_deriv_max'] = np.max(derivative)
    features['first_deriv_min'] = np.min(derivative)

    # --- Frequency-domain features ---
    hist, _ = np.histogram(ppg_series, bins='auto')
    features['entropy'] = entropy(hist)

    return features

print("Utility functions loaded.")



#### 4. Main Data Processing Loop [Example]

This cell downloads data for each patient, applies the filtering and windowing logic, extracts the features, and stores the results.

In [ ]:
# Load clinical information and track list
df_cases = pd.read_csv(VITALDB_DATA_URL)
df_trks = pd.read_csv(VITALDB_TRACKS_URL)

# Filter for cases with demographic and BG data
bg_data_cases = df_cases[df_cases['preop_gluc'].notna() & df_cases['age'].notna()].copy()
caseids_to_process = list(bg_data_cases['caseid'].unique())

print(f"Found {len(caseids_to_process)} cases with relevant data to process.")

aggregated_data = []
processed_case_count = 0

# You can limit the number of cases to process for testing
for caseid in caseids_to_process:
    print(f"Processing Case ID: {caseid}...")

    # Load raw PPG signal for the case
    vals = vitaldb.load_case(caseid, [PPG_SIGNAL_NAME], 1/SAMPLE_RATE_HZ)

    if vals is None or vals.size == 0:
        print(f"  No PPG data found for Case ID {caseid}. Skipping.")
        continue

    ppg_signal = vals[:, 0]

    # Get metadata for the current case
    case_meta = bg_data_cases[bg_data_cases['caseid'] == caseid].iloc[0]

    # Filter signal within the valid time window around 'opstart' for BG stability
    opstart_seconds = case_meta['opstart']
    start_idx = max(0, int((opstart_seconds - VALID_WINDOW_MINUTES * 60) * SAMPLE_RATE_HZ))
    end_idx = min(len(ppg_signal), int((opstart_seconds + VALID_WINDOW_MINUTES * 60) * SAMPLE_RATE_HZ))

    windowed_signal = ppg_signal[start_idx:end_idx]

    if len(windowed_signal) < SAMPLES_PER_WINDOW:
        print(f"  Signal for Case ID {caseid} is too short. Skipping.")
        continue

    # Apply bandpass filter to the entire windowed signal to remove noise
    filtered_signal = butter_bandpass_filter(windowed_signal, lowcut=0.5, highcut=8, fs=SAMPLE_RATE_HZ)

    # Iterate through the filtered signal in non-overlapping windows
    for i in range(0, len(filtered_signal) - SAMPLES_PER_WINDOW + 1, SAMPLES_PER_WINDOW):
        ppg_window = filtered_signal[i:i + SAMPLES_PER_WINDOW]

        # Extract hand-engineered features
        features = extract_ppg_features(ppg_window, SAMPLE_RATE_HZ)

        # Add metadata and target value
        features['caseid'] = caseid
        features['preop_gluc'] = case_meta['preop_gluc']
        features['age'] = case_meta['age']
        features['sex'] = 1 if case_meta['sex'] == 'F' else 0
        features['preop_dm'] = case_meta['preop_dm']
        features['weight'] = case_meta['weight']
        features['height'] = case_meta['height']

        # Store the complete record
        aggregated_data.append(features)

    processed_case_count += 1

print(f"\nProcessed a total of {processed_case_count} cases.")


#### 5. Create DataFrame and Save to CSV [Example]

This final cell converts the list of dictionaries into a pandas DataFrame and saves it to a single CSV file, which is now ready for model training.


In [ ]:
# Convert list of dictionaries to a DataFrame
if aggregated_data:
    final_df = pd.DataFrame(aggregated_data)
    final_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Successfully saved processed data to {OUTPUT_FILE}.")
    print(f"Final DataFrame shape: {final_df.shape}")
else:
    print("No data was aggregated. Please check your data source and processing



## TOGETHER!!. [Note: This worked the last time I ran It]

In [ ]:
import pandas as pd
import vitaldb
import numpy as np
import os
from scipy.signal import find_peaks, butter, lfilter
from scipy.stats import entropy

# --- CONFIGURATION ---
OUTPUT_DIR = "./processed_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_FILE = os.path.join(OUTPUT_DIR, "vitaldb_ppg_extracted_features.csv")

VITALDB_DATA_URL = "https://api.vitaldb.net/cases"
VITALDB_TRACKS_URL = "https://api.vitaldb.net/trks"

PPG_SIGNAL_NAME = 'SNUADC/PLETH'
SAMPLE_RATE_HZ = 100
WINDOW_DURATION_SECONDS = 30
SAMPLES_PER_WINDOW = SAMPLE_RATE_HZ * WINDOW_DURATION_SECONDS
VALID_WINDOW_MINUTES = 8  # +/- window around 'opstart' for BG stability

# --- UTILITY FUNCTIONS ---
def butter_bandpass(lowcut, highcut, fs, order=3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=3):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def extract_ppg_features(ppg_series, fs):
    """
    Extracts a set of robust, hand-engineered features from a single PPG signal window.
    """
    features = {}

    # Clean the data: remove NaN values
    ppg_series_clean = ppg_series[np.isfinite(ppg_series)]

    if ppg_series_clean.size == 0:
        return {
            'ppg_mean': 0, 'ppg_std': 0, 'mean_pp_interval_s': 0,
            'std_pp_interval_s': 0, 'ppg_freq': 0, 'auc': 0,
            'first_deriv_max': 0, 'first_deriv_min': 0, 'entropy': 0
        }

    features['ppg_mean'] = np.mean(ppg_series_clean)
    features['ppg_std'] = np.std(ppg_series_clean)

    peaks, _ = find_peaks(ppg_series_clean, distance=50, height=0)
    if len(peaks) > 1:
        pp_intervals = np.diff(peaks)
        features['mean_pp_interval_s'] = np.mean(pp_intervals) / fs
        features['std_pp_interval_s'] = np.std(pp_intervals) / fs
        features['ppg_freq'] = fs / np.mean(pp_intervals)
    else:
        features['mean_pp_interval_s'] = 0
        features['std_pp_interval_s'] = 0
        features['ppg_freq'] = 0

    features['auc'] = np.trapezoid(ppg_series_clean)

    derivative = np.diff(ppg_series_clean)
    features['first_deriv_max'] = np.max(derivative)
    features['first_deriv_min'] = np.min(derivative)

    hist, _ = np.histogram(ppg_series_clean, bins='auto')
    features['entropy'] = entropy(hist)

    return features

# --- MAIN DATA PROCESSING LOOP ---
print("Starting data processing...")

# Load clinical information and track list
df_cases = pd.read_csv(VITALDB_DATA_URL)
df_trks = pd.read_csv(VITALDB_TRACKS_URL)

# Filter for cases with demographic and BG data
bg_data_cases = df_cases[df_cases['preop_gluc'].notna() & df_cases['age'].notna()].copy()
caseids_to_process = list(bg_data_cases['caseid'].unique())

print(f"Found {len(caseids_to_process)} cases with relevant data to process.")

# Check if the output file already exists to decide whether to write the header
output_file_exists = os.path.exists(OUTPUT_FILE)

for caseid in caseids_to_process:
    print(f"Processing Case ID: {caseid}...")

    # Load raw PPG signal for the case
    vals = vitaldb.load_case(caseid, [PPG_SIGNAL_NAME], 1/SAMPLE_RATE_HZ)

    if vals is None or vals.size == 0:
        print(f"  No PPG data found for Case ID {caseid}. Skipping.")
        continue

    ppg_signal = vals[:, 0]

    # Get metadata for the current case
    case_meta = bg_data_cases[bg_data_cases['caseid'] == caseid].iloc[0]

    # Filter signal within the valid time window around 'opstart' for BG stability
    opstart_seconds = case_meta['opstart']
    start_idx = max(0, int((opstart_seconds - VALID_WINDOW_MINUTES * 60) * SAMPLE_RATE_HZ))
    end_idx = min(len(ppg_signal), int((opstart_seconds + VALID_WINDOW_MINUTES * 60) * SAMPLE_RATE_HZ))

    windowed_signal = ppg_signal[start_idx:end_idx]

    if len(windowed_signal) < SAMPLES_PER_WINDOW:
        print(f"  Signal for Case ID {caseid} is too short. Skipping.")
        continue

    # Apply bandpass filter to the entire windowed signal to remove noise
    filtered_signal = butter_bandpass_filter(windowed_signal, lowcut=0.5, highcut=8, fs=SAMPLE_RATE_HZ)

    # Process and save windows for the current case
    case_data = []
    for i in range(0, len(filtered_signal) - SAMPLES_PER_WINDOW + 1, SAMPLES_PER_WINDOW):
        ppg_window = filtered_signal[i:i + SAMPLES_PER_WINDOW]

        # Extract hand-engineered features
        features = extract_ppg_features(ppg_window, SAMPLE_RATE_HZ)

        row = {
            'caseid': caseid,
            'preop_gluc': case_meta['preop_gluc'],
            'age': case_meta['age'],
            'sex': 1 if case_meta['sex'] == 'F' else 0,
            'preop_dm': case_meta['preop_dm'],
            'weight': case_meta['weight'],
            'height': case_meta['height'],
            **features # Unpack the extracted features
        }
        case_data.append(row)

    # Convert list of dicts to a DataFrame and append to the CSV
    case_df = pd.DataFrame(case_data)

    # Write the header only once at the beginning if the file is new
    header = not output_file_exists
    case_df.to_csv(OUTPUT_FILE, mode='a', header=header, index=False)
    # Set the flag to False after the first write
    output_file_exists = True

print(f"\nProcessing complete. All data saved to {OUTPUT_FILE}.")




## Example DNN training code [Using Pytorch. Need to change this to Tensforflow]

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# ---- CONFIG ----
FILTERED_FILE = '/media/impact/Data/Isaac_BG_SmartWatch/feature_selection/filtered_ppg_features.csv'
CASEID_COL = 'caseid'
TARGET_COL = 'preop_gluc'
BATCH_SIZE = 16
EPOCHS = 80
LEARNING_RATE = 1e-3
DROPOUT = 0.2
DNN_LAYERS = [128, 64, 32]
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Loading filtered dataset...")
df = pd.read_csv(FILTERED_FILE)
df = df.dropna()
print(f"Loaded shape: {df.shape}")

# ----- Feature/target selection -----
features_to_use = [col for col in df.columns if col not in [CASEID_COL, TARGET_COL]]
X = df[features_to_use].values.astype(np.float32)
y = df[TARGET_COL].values.astype(np.float32)
caseids = df[CASEID_COL].values

print(f"Using {len(features_to_use)} features: {features_to_use[:10]} ...")

# ----- Subject-wise train/test split -----
unique_caseids = np.unique(caseids)
train_ids, test_ids = train_test_split(unique_caseids, test_size=0.2, random_state=42)
train_mask = np.isin(caseids, train_ids)
test_mask = ~train_mask
X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]
print(f"Train: {X_train.shape[0]}, Test: {X_test.shape[0]}")

# ----- Fit scaler only on train! -----
x_scaler, y_scaler = MinMaxScaler(), MinMaxScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled  = x_scaler.transform(X_test)
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test_scaled  = y_scaler.transform(y_test.reshape(-1, 1)).flatten()
y_train_orig, y_test_orig = y_train, y_test

# ----- PyTorch DataLoader -----
X_train_t = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_t = torch.tensor(y_train_scaled, dtype=torch.float32).unsqueeze(1)
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_t = torch.tensor(y_test_scaled, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset  = TensorDataset(X_test_t, y_test_t)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# ----- DNN Model -----
class DNNRegressor(nn.Module):
    def __init__(self, in_features, layers, dropout):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(in_features, layers[0]),
            nn.ReLU(),
            nn.BatchNorm1d(layers[0]),
            nn.Dropout(dropout),
            nn.Linear(layers[0], layers[1]),
            nn.ReLU(),
            nn.BatchNorm1d(layers[1]),
            nn.Dropout(dropout),
            nn.Linear(layers[1], layers[2]),
            nn.ReLU(),
            nn.Linear(layers[2], 1)
        )
    def forward(self, x):
        return self.seq(x)

model = DNNRegressor(X_train_t.shape[1], DNN_LAYERS, DROPOUT).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

# ----- Training Loop -----
train_losses, val_losses = [], []
print("Training DNN model...")
for epoch in range(EPOCHS):
    model.train()
    epoch_train_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item() * xb.size(0)
    epoch_train_loss /= len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            pred = model(xb)
            val_loss += criterion(pred, yb).item() * xb.size(0)
    val_loss /= len(test_loader.dataset)
    val_losses.append(val_loss)
    if (epoch+1) % 10 == 0 or epoch < 5:
        print(f"Epoch {epoch+1}: train {epoch_train_loss:.4f} val {val_loss:.4f}")

# ----- Evaluation -----
model.eval()
y_pred_scaled = []
with torch.no_grad():
    for xb, _ in test_loader:
        xb = xb.to(DEVICE)
        preds = model(xb).cpu().numpy().flatten()
        y_pred_scaled.append(preds)
y_pred_scaled = np.concatenate(y_pred_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
mae = mean_absolute_error(y_test_orig, y_pred)
mape = mean_absolute_percentage_error(y_test_orig, y_pred) * 100
print(f"\nTest MAE: {mae:.2f} mg/dL")
print(f"Test MAPE: {mape:.2f}%")

# ----- Plots -----
plt.figure(figsize=(8, 4))
plt.plot(train_losses, label='Train Loss (MSE)')
plt.plot(val_losses, label='Val Loss (MSE)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend(); plt.tight_layout(); plt.show()

plt.figure(figsize=(7, 7))
plt.scatter(y_test_orig, y_pred, alpha=0.5, label='Test Predictions')
slope, intercept = np.polyfit(y_test_orig, y_pred, 1)
plt.plot([y_test_orig.min(), y_test_orig.max()], [y_test_orig.min(), y_test_orig.max()], 'k--', label='Ideal (y=x)')
plt.plot([y_test_orig.min(), y_test_orig.max()],
         [slope * y_test_orig.min() + intercept, slope * y_test_orig.max() + intercept],
         color='red', lw=2, label=f'Trendline (slope={slope:.2f})')
plt.xlabel("Actual BG (mg/dL)")
plt.ylabel("Predicted BG (mg/dL)")
plt.title("Actual vs. Predicted BG (PyTorch DNN)")
plt.legend(); plt.tight_layout(); plt.show()
print(f"Scatter plot trendline slope: {slope:.2f}")

# Save PyTorch model
save_path = '/media/impact/Data/Isaac_BG_SmartWatch/bg_best_dnn_model2.pt'
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

